In [58]:
import pandas as pd
import numpy as np

import json
import requests
from IPython.display import clear_output

import psycopg2 as pg
conn = pg.connect(database = "discoverify_db",
                  user="postgres", 
                  password="damara1004")

conn.autocommit = True

def run_query(q):
    with conn:
        try:
            cur = conn.cursor()
            cur.execute(q)
            return pd.read_sql(q, conn)

        except (Exception, pg.DatabaseError) as error:
            print(error)

def run_command(c):
    with conn:
        try:
            cur = conn.cursor()
            cur.execute(c)
            cur.close()
            conn.commit()
            
        except (Exception, pg.DatabaseError) as error:
            print(error)
            
# reading in created datasets
# track_metadata = pd.read_csv('../data/all_tracks_cleaned_V2.csv', index_col=0)
# spotify_features = pd.read_csv('../data/audio_features_final.csv', index_col=0)

In [6]:
c = '''
    CREATE DATABASE discoverify_db;
    '''

run_command(c)

c = '''
    CREATE TABLE IF NOT EXISTS metadata (
        track_id VARCHAR(22) PRIMARY KEY,
        track_name TEXT NOT NULL,
        artist TEXT NOT NULL,
        artist_id VARCHAR(22) NOT NULL,
        genre_1 TEXT NOT NULL,
        genre_2 TEXT,
        genre_3 TEXT
        );
        
    '''

run_command(c)

c = '''
    CREATE TABLE IF NOT EXISTS spotify_features (
        id VARCHAR(22) PRIMARY KEY,
        danceability NUMERIC,
        energy NUMERIC,
        loudness NUMERIC,
        speechiness NUMERIC,
        acousticness NUMERIC,
        instrumentalness NUMERIC,
        liveness NUMERIC,
        valence NUMERIC,
        tempo NUMERIC,
        
        FOREIGN KEY (id) REFERENCES metadata (track_id)
        );
    '''

run_command(c)

c = '''
    CREATE TABLE IF NOT EXISTS librosa_features (
        track_id VARCHAR(22) PRIMARY KEY,
        spectral_centroid NUMERIC,
        spectral_bandwidth NUMERIC,
        rolloff NUMERIC,
        zero_crossing_rate NUMERIC,
        mfcc1 NUMERIC,
        mfcc2 NUMERIC,
        mfcc3 NUMERIC,
        mfcc4 NUMERIC,
        mfcc5 NUMERIC,
        mfcc6 NUMERIC,
        mfcc7 NUMERIC,
        mfcc8 NUMERIC,
        mfcc9 NUMERIC,
        mfcc10 NUMERIC,
        mfcc11 NUMERIC,
        mfcc12 NUMERIC,
        mfcc13 NUMERIC,
        mfcc14 NUMERIC,
        mfcc15 NUMERIC,
        mfcc16 NUMERIC,
        mfcc17 NUMERIC,
        mfcc18 NUMERIC,
        mfcc19 NUMERIC,
        mfcc20 NUMERIC,
        C NUMERIC,
        "C#" NUMERIC,
        D NUMERIC,
        "D#" NUMERIC,
        E NUMERIC,
        F NUMERIC,
        "F#" NUMERIC,
        G NUMERIC,
        "G#" NUMERIC,
        A NUMERIC,
        "A#" NUMERIC,
        B NUMERIC,

        FOREIGN KEY (track_id) REFERENCES metadata (track_id)
        );
'''

run_command(c)

In [15]:
metadata_cols = ['track_id','track_name','artist','artist_id','genre_1','genre_2','genre_3']

metadata = track_metadata[metadata_cols]

to_insert = list(metadata.itertuples(index=False, name=None))

# for i,row in enumerate(to_insert):
#     c = f'''INSERT INTO metadata ({row});'''
    
#     run_command(c)
    
#     clear_output(wait=True)
#     print(f"{i+1}/{len(to_insert)}")

In [11]:
metadata = pd.read_csv('../data/metadata.csv', sep=";")

,Unnamed: 0,Unnamed: 0.1,track_id,track_name,artist,artist_id,genre_1,genre_2,genre_3
0,0,0,6YDf6QV7QfCEo8O2dbWalK,Real Love Baby,Father John Misty,2kGBy2WHvF0VdZyqiVCkDT,art pop,chamber pop,freak folk
1,1,1,0qaWxVVd3KrP4WY27KWpIe,In a River,Rostam,04XggbrM51GcFPTxBYtRXT,art pop,chamber pop,indie pop
2,2,2,1yZMv2GMAibgLGsQiowZrt,Nancy From Now On,Father John Misty,2kGBy2WHvF0VdZyqiVCkDT,art pop,chamber pop,freak folk
3,3,3,2eg2gvPXuwZ9FyrPaLgrXi,Chateau Lobby #4 (in C for Two Virgins),Father John Misty,2kGBy2WHvF0VdZyqiVCkDT,art pop,chamber pop,freak folk
4,4,4,0wzruvvN7f5wu39aFcjTMw,Hollywood Forever Cemetery Sings,Father John Misty,2kGBy2WHvF0VdZyqiVCkDT,art pop,chamber pop,freak folk


In [14]:
metadata = metadata[metadata.columns[2:]]
metadata

,track_id,track_name,artist,artist_id,genre_1,genre_2,genre_3
0,6YDf6QV7QfCEo8O2dbWalK,Real Love Baby,Father John Misty,2kGBy2WHvF0VdZyqiVCkDT,art pop,chamber pop,freak folk
1,0qaWxVVd3KrP4WY27KWpIe,In a River,Rostam,04XggbrM51GcFPTxBYtRXT,art pop,chamber pop,indie pop
2,1yZMv2GMAibgLGsQiowZrt,Nancy From Now On,Father John Misty,2kGBy2WHvF0VdZyqiVCkDT,art pop,chamber pop,freak folk
3,2eg2gvPXuwZ9FyrPaLgrXi,Chateau Lobby #4 (in C for Two Virgins),Father John Misty,2kGBy2WHvF0VdZyqiVCkDT,art pop,chamber pop,freak folk
4,0wzruvvN7f5wu39aFcjTMw,Hollywood Forever Cemetery Sings,Father John Misty,2kGBy2WHvF0VdZyqiVCkDT,art pop,chamber pop,freak folk
...,...,...,...,...,...,...,...
230451,6jVVnTIC1zHFFdn2rBOE2g,This Must Be the Place (Naive Melody) - Live,David Byrne,20vuBdFblWUo2FCOvUzusB,art pop,art punk,art rock
230452,3xJ8siSD7r5mDqLNtQ5CPf,A Soft Seduction,David Byrne,20vuBdFblWUo2FCOvUzusB,art pop,art punk,art rock
230453,6Mc0SvjQ2By4ZJWdF77M2C,Dance on Vaseline,David Byrne,20vuBdFblWUo2FCOvUzusB,art pop,art punk,art rock
230454,392cd1euczCOcZAs0SvvZS,The Last Emperor (Main Title Theme),David Byrne,20vuBdFblWUo2FCOvUzusB,art pop,art punk,art rock


In [15]:
metadata.to_csv('../data/metadata_falseindex.csv', sep=";", index=False)

In [22]:
tuple(metadata.iloc[1,:].values)

('0qaWxVVd3KrP4WY27KWpIe',
 'In a River',
 'Rostam',
 '04XggbrM51GcFPTxBYtRXT',
 'art pop',
 'chamber pop',
 'indie pop')

In [61]:
for i in range(20,30):
    record = metadata.iloc[i,:].values
    
    new_record = tuple(elem if str(elem)!='nan' else None for elem in record)
        
    
    print(new_record)

('2CPLGREl5ahzA2WPBMkpxh', 'Fallingwater', 'Maggie Rogers', '4NZvixzsSefsNiIqXn0NDe', 'electropop', 'pop', None)
('4HfLQJtVT1KiX1eVedDyTm', 'Alaska', 'Maggie Rogers', '4NZvixzsSefsNiIqXn0NDe', 'electropop', 'pop', None)
('68vxG1i6RSmrsY9mRdqDgG', 'Burning', 'Maggie Rogers', '4NZvixzsSefsNiIqXn0NDe', 'electropop', 'pop', None)
('6PyZGb1rB7oLKCICWTmGYa', 'Say It', 'Maggie Rogers', '4NZvixzsSefsNiIqXn0NDe', 'electropop', 'pop', None)
('3D4uUFRoYDFHbkQlTKL2Om', 'Give A Little', 'Maggie Rogers', '4NZvixzsSefsNiIqXn0NDe', 'electropop', 'pop', None)
('5sn1BmEOPP1bu4cZq9OgoF', 'Tim McGraw - Recorded At Spotify Studios NYC', 'Maggie Rogers', '4NZvixzsSefsNiIqXn0NDe', 'electropop', 'pop', None)
('4L9l5x0WuzNtyEBeqzvgPE', 'Overnight', 'Maggie Rogers', '4NZvixzsSefsNiIqXn0NDe', 'electropop', 'pop', None)
('5RRNZFyOi17nTh2bPEKPtp', 'Dog Years', 'Maggie Rogers', '4NZvixzsSefsNiIqXn0NDe', 'electropop', 'pop', None)
('7oJmIBxEAYjfkS7lhZ7M1S', 'Sometimes', 'Goth Babe', '7o96HO2zrujyATtVsqGhh3', 'indie 

In [63]:
cur = conn.cursor()

for i in range(20,30):
    record = tuple(metadata.iloc[i,:].values)
    
    new_record = tuple(elem if str(elem)!='nan' else None for elem in record)
    
    try:
        cur.execute("INSERT INTO metadata (track_id, track_name, artist, artist_id, genre_1, genre_2, genre_3) VALUES (%s, %s, %s, %s, %s, %s, %s)", record)
        conn.commit()
        
    except:
        pass


In [40]:
cur.execute("INSERT INTO metadata (track_id, track_name, artist, artist_id, genre_1, genre_2, genre_3) VALUES (%s, %s, %s, %s, %s, %s, %s)", record)
conn.commit()

"I''m Writing a Novel"

In [41]:
test_2 = 'Hello'

"'" in test_2

False